In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics.pairwise import paired_distances
from sklearn.preprocessing import StandardScaler
import csv
from sklearn.model_selection import GridSearchCV
from sklearn.metrics._scorer import make_scorer
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import ParameterGrid
from scipy.stats import zscore

In [3]:
def custom_loss_func(ground_truth, predictions):
     return np.sum(paired_distances(ground_truth, predictions))/len(ground_truth)

In [8]:
df = pd.read_csv('development.csv', sep=',')
y = df[['x','y']].values
df.drop(columns=['x','y'], inplace=True)

In [4]:
from sklearn.decomposition import PCA #132.61
pca=PCA()
pca_df = pca.fit_transform(df.values)
mdf = np.sum(np.cumsum(pca.explained_variance_ratio_) < .90) + 1
pca_m = PCA(n_components=mdf)
pca_df = pca_m.fit_transform(df.values)
X_train = pca_df
y_train = y

df_eval = pd.read_csv('evaluation.csv', sep=',')
eval = df_eval.drop(columns='Id').values

pca_df_eval = pca_m.fit_transform(eval)
X_valid = pca_df_eval

X_train = np.where(np.isinf(X_train), np.finfo(np.float32).max, X_train)
reg = RandomForestRegressor(n_estimators = 200, criterion='squared_error', max_features='sqrt', random_state=42, n_jobs=-1)
reg.fit(X_train, y_train)   
y_pred = reg.predict(X_valid)

header = ['Id', 'Predicted']
with open("submission.csv", 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for i in df_eval['Id']:
        writer.writerow([i, ''.join((str(round(y_pred[i, 0],2)), '|', str(round(y_pred[i, 1],2))))])

In [5]:
from sklearn.neighbors import KNeighborsRegressor #183.341
from sklearn.preprocessing import StandardScaler
#std = StandardScaler()
#std.fit(X_train)
#X_train_norm = std.transform(X_train)
#X_valid_norm = std.transform(X_valid)
reg = KNeighborsRegressor(n_neighbors=7 , weights='distance')
reg.fit(X_train, y_train)                    
y_pred = reg.predict(X_valid)
header = ['Id', 'Predicted']
with open("submission.csv", 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for i in df_eval['Id']:
        writer.writerow([i, ''.join((str(round(y_pred[i, 0],2)), '|', str(round(y_pred[i, 1],2))))])

In [9]:
from sklearn.feature_selection import SelectKBest, f_classif
noise_pmax = []
noise_negpmax = []
noise_area = []#4.7
noise_tmax = []
noise_rms = []
noise_vpp = []
noise_slewrate = []
pmax = [f'pmax[{i}]' for i in range(18)]
negpmax = [f'negpmax[{i}]' for i in range(18)]
#for i in range(0,18):
#    condition = abs(df[f"tmax[{i}]"]) > 1e-6
#    df[f"slewrate[{i}]"] = np.where(condition, ((df[f"pmax[{i}]"] - df[f"negpmax[{i}]"])) / df[f"tmax[{i}]"],  np.finfo(np.float32).max)
df = df[df['negpmax[1]'] < 0]
df = df[df['negpmax[2]'] < 0]
df = df[df['negpmax[3]'] < 0]
df = df[df['negpmax[4]'] < 0]
df = df[df['negpmax[5]'] < 0]
df = df[df['negpmax[6]'] < 0]
df = df[df['negpmax[8]'] < 0]
df = df[df['negpmax[9]'] < 0]
df = df[df['negpmax[10]'] < 0]
df = df[df['negpmax[11]'] < 0]
df = df[df['negpmax[13]'] < 0]
df = df[df['negpmax[14]'] < 0]
df = df[df['negpmax[15]'] < 0]
#df = df[df[negpmax] < 0]
for i in range(0,18):
    df[f"vpp[{i}]"] = (df[f"pmax[{i}]"] - df[f"negpmax[{i}]"])
#for i in range(0,18):
#    df[f"widh[{i}]"] = df[f"area[{i}]"]/((df[f"pmax[{i}]"]-df[f"negpmax[{i}]"])/2)
#for i in range(0,18):
#   df[f"widhRMS[{i}]"] = df[f"rms[{i}]"]/((df[f"pmax[{i}]"]-df[f"negpmax[{i}]"])/2)
for i in [0, 7, 12, 16, 17]: 
    noise_pmax.append(f'pmax[{i}]')
    noise_area.append(f'area[{i}]')
    noise_area.append(f'negpmax[{i}]')
    noise_vpp.append(f'vpp[{i}]')
    #noise_vpp.append(f'slewrate[{i}]')
    #noise_negpmax.append(f'widh[{i}]')
    #noise_slewrate.append(f'slewrate[{i}]')
    #noise_slewrate.append(f'widh[{i}]')


#noise_area.append('area[15]')
#noise_area.append('negpmax[15]')

for i in range(0, 18): 
    noise_rms.append(f'rms[{i}]')
    noise_tmax.append(f'tmax[{i}]')
    #noise_negpmax.append(f'negpmax[{i}]')
    #noise_area.append(f'area[{i}]')
noise = noise_pmax + noise_negpmax + noise_area + noise_tmax + noise_rms+ noise_vpp + noise_slewrate
df.drop(columns=noise, inplace=True)

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(df, y, test_size=0.2, random_state=42)

In [ ]:
df_eval = pd.read_csv('evaluation.csv', sep=',')#4.76 con -neg
#for i in range(0,18):
#    condition = abs(df_eval[f"tmax[{i}]"]) > 1e-6
#    df_eval[f"slewrate[{i}]"] = np.where(condition, ((df_eval[f"pmax[{i}]"] - df_eval[f"negpmax[{i}]"])) / df_eval[f"tmax[{i}]"], np.finfo(np.float32).max)
for i in range(0,18):
    df_eval[f"vpp[{i}]"] = (df_eval[f"pmax[{i}]"] - df_eval[f"negpmax[{i}]"])
#for i in range(0,18):
#    df_eval[f"widh[{i}]"] = df_eval[f"area[{i}]"]/((df_eval[f"pmax[{i}]"]-df_eval[f"negpmax[{i}]"])/2)
#for i in range(0,18):
#    df_eval[f"widhRMS[{i}]"] = df_eval[f"rms[{i}]"]/((df_eval[f"pmax[{i}]"]-df_eval[f"negpmax[{i}]"])/2)
df_eval.drop(columns=noise, inplace=True)

X_train = df.drop(columns=['x', 'y']).values
y_trainX = df[['x']].values
y_trainY = df[['y']].values
y_train = df[['x','y']].values
X_test = df_eval.drop(columns='Id').values

Random Forest con (X,Y)

In [7]:
reg = RandomForestRegressor(n_estimators = 200, criterion='squared_error', max_features='sqrt', random_state=42, n_jobs=-1)
reg.fit(X_train, y_train)                    
y_pred = reg.predict(X_valid) 
print(custom_loss_func(y_valid,y_pred))

3.930409975524837


Random Forest (X) + Random Forest(Y)

In [8]:
y_train_x = y_train[:,0]
y_train_y = y_train[:,1]
#yy = df[['x']].values
reg = RandomForestRegressor(n_estimators = 200, criterion='squared_error', max_features='sqrt', random_state=42, n_jobs=-1)
reg.fit(X_train, y_train_x)                    
y_pred_x = reg.predict(X_valid) 
reg = RandomForestRegressor(n_estimators = 200, criterion='squared_error', max_features='sqrt', random_state=42, n_jobs=-1)
reg.fit(X_train, y_train_y)                    
y_pred_y = reg.predict(X_valid)
y_pred_y=y_pred_y.reshape(-1,1)
y_pred_x = y_pred_x.reshape(-1,1)
y_pred= np.hstack((y_pred_x,y_pred_y))
print(custom_loss_func(y_valid,y_pred))


4.177011414542949


k-ne

In [9]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
std.fit(X_train)
X_train_norm = std.transform(X_train)
X_valid_norm = std.transform(X_valid)
reg = KNeighborsRegressor(n_neighbors=7 , weights='distance')
reg.fit(X_train_norm, y_train)                    
y_pred = reg.predict(X_valid_norm)
print(custom_loss_func(y_valid,y_pred))

7.488862367515643


In [4]:
from sklearn.feature_selection import SelectKBest, f_classif
df = pd.read_csv('development.csv', sep=',')

In [5]:
noise_pmax = []
noise_negpmax = []
noise_area = []#4.7
noise_tmax = []
noise_rms = []
noise_vpp = []
noise_slewrate = []
pmax = [f'pmax[{i}]' for i in range(18)]
negpmax = [f'negpmax[{i}]' for i in range(18)]
for i in range(0,18):
    df[f"vpp[{i}]"] = (df[f"pmax[{i}]"] - df[f"negpmax[{i}]"])
df = df[df['negpmax[1]'] < 0]
df = df[df['negpmax[2]'] < 0]
df = df[df['negpmax[3]'] < 0]
df = df[df['negpmax[4]'] < 0]
df = df[df['negpmax[5]'] < 0]
df = df[df['negpmax[6]'] < 0]
df = df[df['negpmax[8]'] < 0]
df = df[df['negpmax[9]'] < 0]
df = df[df['negpmax[10]'] < 0]
df = df[df['negpmax[11]'] < 0]
df = df[df['negpmax[13]'] < 0]
df = df[df['negpmax[14]'] < 0]
df = df[df['negpmax[15]'] < 0]
for i in [0, 7, 12, 16, 17]: 
    noise_pmax.append(f'pmax[{i}]')
    noise_area.append(f'area[{i}]')
    noise_area.append(f'negpmax[{i}]')
    noise_vpp.append(f'vpp[{i}]')
for i in range(0, 18): 
    noise_rms.append(f'rms[{i}]')
    noise_tmax.append(f'tmax[{i}]')
noise = noise_pmax + noise_negpmax + noise_area + noise_tmax + noise_rms+ noise_vpp + noise_slewrate
df.drop(columns=noise, inplace=True)
y = df[['x','y']].values
df.drop(columns=['x','y'], inplace=True)

In [6]:
df_eval = pd.read_csv('evaluation.csv', sep=',')#4.76 con -neg
#for i in range(0,18):
#    condition = abs(df_eval[f"tmax[{i}]"]) > 1e-6
#    df_eval[f"slewrate[{i}]"] = np.where(condition, ((df_eval[f"pmax[{i}]"] - df_eval[f"negpmax[{i}]"])) / df_eval[f"tmax[{i}]"], np.finfo(np.float32).max)
for i in range(0,18):
    df_eval[f"vpp[{i}]"] = (df_eval[f"pmax[{i}]"] - df_eval[f"negpmax[{i}]"])
#for i in range(0,18):
#    df_eval[f"widh[{i}]"] = df_eval[f"area[{i}]"]/((df_eval[f"pmax[{i}]"]-df_eval[f"negpmax[{i}]"])/2)
#for i in range(0,18):
#    df_eval[f"widhRMS[{i}]"] = df_eval[f"rms[{i}]"]/((df_eval[f"pmax[{i}]"]-df_eval[f"negpmax[{i}]"])/2)
df_eval.drop(columns=noise, inplace=True)

X_train_valid = df.values #df.drop(columns=['x', 'y']).values
y_train_valid = y#df[['x','y']].values
X_test = df_eval.drop(columns='Id').values

In [25]:
custom_scorer = make_scorer(custom_loss_func, greater_is_better=False)
param_grid = {'n_estimators': [50, 100, 200],
              'criterion': ['squared_error'],
              'max_features': ['sqrt', 'log2'],
              'random_state': [42],
              'n_jobs': [-1]      
}
#X=df.values
#X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42) #stratify=y
gs = GridSearchCV(RandomForestRegressor(), param_grid, scoring=custom_scorer, n_jobs=-1, cv=5)
gs.fit(X_train_valid, y_train_valid)
print(gs.best_score_)   # -4.823035566366996
print(gs.best_params_)  # {'criterion': 'squared_error', 'max_features': 'sqrt', 'n_estimators': 50, 'n_jobs': -1, 'random_state': 42}
y_pred = gs.predict(X_test)
#print(f'Metric value: {custom_loss_func(y_test, y_pred)}') # Metric value: 4.742701679372149

-3.9764732556909985
{'criterion': 'squared_error', 'max_features': 'sqrt', 'n_estimators': 200, 'n_jobs': -1, 'random_state': 42}
Metric value: 3.9121012047690993


In [28]:
df = pd.read_csv('development.csv', sep=',')
y = df[['x','y']].values
df.drop(columns=['x','y'], inplace=True)
from sklearn.decomposition import PCA
pca=PCA()
pca_df = pca.fit_transform(df.values)
mdf = np.sum(np.cumsum(pca.explained_variance_ratio_) < .90) + 1
pca_m = PCA(n_components=mdf)
pca_df = pca_m.fit_transform(df.values)
#X_train = pca_df
#y_train = y

X_train, X_valid, y_train, y_valid = train_test_split(pca_df, y, test_size=0.2, random_state=42)

X_train = np.where(np.isinf(X_train), np.finfo(np.float32).max, X_train)
from sklearn.neighbors import KNeighborsRegressor #183.341
from sklearn.preprocessing import StandardScaler

reg = KNeighborsRegressor(n_neighbors=9 , weights='distance', algorithm='auto')
reg.fit(X_train, y_train)                    
y_pred = reg.predict(X_valid)

print(f'Metric value: {custom_loss_func(y_valid, y_pred)}') # Metric value: 4.742701679372149

Metric value: 24.713063298177318


In [36]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
df = pd.read_csv('development.csv', sep=',')

noise_pmax = []
noise_negpmax = []
noise_area = []#4.7
noise_tmax = []
noise_rms = []
noise_vpp = []
noise_slewrate = []
pmax = [f'pmax[{i}]' for i in range(18)]
negpmax = [f'negpmax[{i}]' for i in range(18)]
for i in range(0,18):
    df[f"vpp[{i}]"] = (df[f"pmax[{i}]"] - df[f"negpmax[{i}]"])
df = df[df['negpmax[1]'] < 0]
df = df[df['negpmax[2]'] < 0]
df = df[df['negpmax[3]'] < 0]
df = df[df['negpmax[4]'] < 0]
df = df[df['negpmax[5]'] < 0]
df = df[df['negpmax[6]'] < 0]
df = df[df['negpmax[8]'] < 0]
df = df[df['negpmax[9]'] < 0]
df = df[df['negpmax[10]'] < 0]
df = df[df['negpmax[11]'] < 0]
df = df[df['negpmax[13]'] < 0]
df = df[df['negpmax[14]'] < 0]
df = df[df['negpmax[15]'] < 0]
for i in [0, 7, 12, 16, 17]: 
    noise_pmax.append(f'pmax[{i}]')
    noise_area.append(f'area[{i}]')
    noise_area.append(f'negpmax[{i}]')
    noise_vpp.append(f'vpp[{i}]')
for i in range(0, 18): 
    noise_rms.append(f'rms[{i}]')
    noise_tmax.append(f'tmax[{i}]')
noise = noise_pmax + noise_negpmax + noise_area + noise_tmax + noise_rms+ noise_vpp + noise_slewrate
df.drop(columns=noise, inplace=True)
y = df[['x','y']].values
#df.drop(columns=['x','y'], inplace=True)
df_eval = pd.read_csv('evaluation.csv', sep=',')#4.76 con -neg
#for i in range(0,18):
#    condition = abs(df_eval[f"tmax[{i}]"]) > 1e-6
#    df_eval[f"slewrate[{i}]"] = np.where(condition, ((df_eval[f"pmax[{i}]"] - df_eval[f"negpmax[{i}]"])) / df_eval[f"tmax[{i}]"], np.finfo(np.float32).max)
for i in range(0,18):
    df_eval[f"vpp[{i}]"] = (df_eval[f"pmax[{i}]"] - df_eval[f"negpmax[{i}]"])
#for i in range(0,18):
#    df_eval[f"widh[{i}]"] = df_eval[f"area[{i}]"]/((df_eval[f"pmax[{i}]"]-df_eval[f"negpmax[{i}]"])/2)
#for i in range(0,18):
#    df_eval[f"widhRMS[{i}]"] = df_eval[f"rms[{i}]"]/((df_eval[f"pmax[{i}]"]-df_eval[f"negpmax[{i}]"])/2)
df_eval.drop(columns=noise, inplace=True)

X_train = df.drop(columns=['x', 'y']).values
y_trainX = df[['x']].values
y_trainY = df[['y']].values
y_train = df[['x','y']].values
X_test = df_eval.drop(columns='Id').values

reg = RandomForestRegressor(n_estimators = 200, criterion='squared_error', max_features='sqrt', random_state=42, n_jobs=-1)
reg.fit(X_train, y_train)                    
y_pred = reg.predict(X_valid) 

#y = df[['x','y']].values
# Crea un DataFrame di esempio con due colonne come coordinate

# Crea una figura 3D
fig = plt.figure(figsize=(8, 6))
plt.scatter(y_pred[:,0], y_pred[:,1], c='r', marker='o', alpha=0.25)
#plt.scatter(y[:,0], y[:,1], c='r', marker='o', alpha=0.25)
# Aggiungi etichette agli as

# Mostra il plot
plt.show()


ValueError: X has 14 features, but RandomForestRegressor is expecting 52 features as input.